In [1]:
#!pip uninstall keras -y
#!pip uninstall keras-nightly -y
#!pip uninstall keras-Preprocessing -y
#!pip uninstall keras-vis -y
#!pip uninstall tensorflow -y
#!pip install tensorflow
#!pip install keras 
#!pip install wordcloud

In [2]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam#Try all of these
from sklearn.preprocessing import LabelEncoder

print(tf.__version__)

2021-07-31 17:15:07.121637: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


2.4.1


In [47]:
#!wget "https://s3-ap-southeast-1.amazonaws.com/he-public-data/dataset52a7b21.zip"
#!unzip dataset52a7b21.zip

In [48]:
#len(train_df)=2903024

2903024

In [49]:
train_df=pd.read_csv('dataset/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [50]:
x = train_df['TITLE'].astype('str')

In [51]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
 raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2021-07-31 15:21:12.494732: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 15:21:12.495353: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-07-31 15:21:12.495409: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-31 15:21:12.495445: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-07-31 15:21:12.495466: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-07-31 15:21:12.495485: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2021-07-

In [53]:
le = LabelEncoder()
y=le.fit_transform(train_df['BROWSE_NODE_ID'])
#y=np.expand_dims(y, axis=1)

In [54]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [56]:
max_features = 20000
max_text_length = 400 #find the text with max length in code and change it for each column

In [57]:
x_tokenizer =text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
print("Happened")
x_tokenized = x_tokenizer.texts_to_sequences(x)
print("Happened")
x_train_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

Happened
Happened


In [58]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [59]:
embedding_dim = 300
embedding_index = dict()
f = open('glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embedding_index))

Loaded 400000 word vectors.


In [60]:
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, index in x_tokenizer.word_index.items():
  if index>max_features -1:
    break
  else:
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[index] = embedding_vector
del(embedding_vector)

In [89]:
embedding_matrix=np.float32(embedding_matrix)

In [62]:
import pickle
dbfile = open('embeddingmatrix.pkl','ab')
pickle.dump(embedding_matrix, dbfile)

In [90]:
embedding_dim = 300
max_features = 20000
max_text_length = 300

In [91]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val,
                                                  y,
                                                  random_state=1,
                                                  test_size=0.15)

In [95]:
model = Sequential()
model.add(Embedding(max_features,
                    embedding_dim,
                    embeddings_initializer=tf.keras.initializers.Constant(
                        embedding_matrix)))#,trainable=False
model.add(Dropout(0.5))


In [96]:
no_of_classes=len(train_df.BROWSE_NODE_ID.unique())
no_of_classes

9150

In [97]:
model.add(Conv1D(500,
                 3,
                 padding='valid'))
model.add(GlobalMaxPooling1D())
model.add(Dense(12000, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(no_of_classes,activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 300)         6000000   
_________________________________________________________________
dropout_11 (Dropout)         (None, None, 300)         0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, None, 500)         450500    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 500)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 12000)             6012000   
_________________________________________________________________
dropout_12 (Dropout)         (None, 12000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 9150)             

In [98]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Train Model

In [99]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [39]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

Num GPUs Available:  1


In [ ]:
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits (logits-model, labels-y)) 
#optimizer = tf.train. AdamOptimizer (learning_rate learning_rate).minimize(cost)

In [101]:
batch_size = 16
epochs = 1

#callbacks = [EarlyStopping(monitor='val_loss')]
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=5*batch_size)
model.save_weights(checkpoint_path.format(epoch=0))

hist_adam=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_val,y_val),
          callbacks=[cp_callback])

 640/5997 [==>...........................] - ETA: 22:06 - loss: 5.8218 - accuracy: 0.2479
Epoch 00001: saving model to training_1/cp.ckpt
1280/5997 [=====>........................] - ETA: 20:34 - loss: 4.9329 - accuracy: 0.3276
Epoch 00001: saving model to training_1/cp.ckpt
1920/5997 [========>.....................] - ETA: 18:00 - loss: 4.5129 - accuracy: 0.3681
Epoch 00001: saving model to training_1/cp.ckpt
2560/5997 [===========>..................] - ETA: 15:16 - loss: 4.2530 - accuracy: 0.3941
Epoch 00001: saving model to training_1/cp.ckpt
3200/5997 [===============>..............] - ETA: 12:28 - loss: 4.0716 - accurac

In [ ]:
import matplotlib.pyplot as plt
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.plot(hist_adam.history['loss'], color='b', label='Training Loss')
plt.plot(hist_adam.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

# Run all the cells above before you sleep

# Run all the cells below after the training is completed

In [3]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

2021-07-31 17:15:31.848560: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-31 17:15:31.849641: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-07-31 17:15:31.905310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0001:00:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-07-31 17:15:31.905364: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-07-31 17:15:31.908555: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-07-31 17:15:31.908613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-07-31

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         6000000   
_________________________________________________________________
dropout_7 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 500)         450500    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 500)         0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, None, 250)         625250    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 250)         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 100)        

In [4]:
test_df=pd.read_csv('dataset/test.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)
x = test_df['TITLE'].astype('str')

In [6]:
max_features = 20000
max_text_length = 400
x_tokenizer =text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
print("Happened")
x_tokenized = x_tokenizer.texts_to_sequences(x)
print("Happened")
x_test_val = sequence.pad_sequences(x_tokenized, maxlen=max_text_length)

Happened
Happened


In [7]:
x_test_val=np.float32(x_test_val)
x_test_val

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 5.4000e+01, 1.7000e+01,
        1.8072e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 6.0000e+00, 3.0000e+01,
        2.7300e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.4100e+02, 5.6000e+01,
        2.7450e+03],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.7000e+01, 1.0000e+01,
        8.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.0000e+01, 3.4800e+02,
        1.8400e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 1.7000e+01, 1.0000e+01,
        1.4000e+01]], dtype=float32)

In [8]:
pred1=new_model.predict(x_test_val)

2021-07-31 17:19:32.725537: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-31 17:19:32.726066: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2596985000 Hz
2021-07-31 17:19:32.846084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-07-31 17:19:33.039336: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-07-31 17:19:33.051071: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-07-31 17:19:33.997781: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-07-31 17:19:34.070991: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform 

In [19]:
sub=pd.read_csv('dataset/sample_submission.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

In [40]:
len(pred1[3])
train_df=pd.read_csv('dataset/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)
le = LabelEncoder()
y=le.fit_transform(train_df['BROWSE_NODE_ID'])

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [41]:
type(y)

numpy.ndarray

In [55]:
pred=[]
for item in range(len(pred1)):
    ind = np.argpartition(pred1[item], -5)[-5:]
    pred.append([ind[np.argsort(pred1[1,ind])][-1]])

In [34]:
#ind = np.argpartition(pred1[1,:], -5)[-5:]
#ind[np.argsort(pred1[1,ind])][-1] #, pred1[1,:]
#pred1[1,ind]#np.max(pred1[1,ind])
#np.argmax(pred1[1,:])

107

In [60]:
pred = [item for sublist in pred for item in sublist]
#pred

[4,
 107,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 641,
 4,
 4,
 2905,
 1043,
 4,
 905,
 890,
 299,
 4,
 299,
 4,
 855,
 65,
 4,
 4,
 413,
 4,
 4,
 4,
 4,
 4,
 944,
 4,
 677,
 4,
 4,
 4,
 335,
 4,
 743,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1075,
 4,
 743,
 4,
 903,
 4,
 4,
 4,
 4,
 369,
 63,
 279,
 4,
 4,
 935,
 944,
 4,
 1200,
 4,
 4,
 4,
 4,
 494,
 4,
 683,
 145,
 64,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 47,
 754,
 4,
 903,
 903,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 911,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 10,
 4,
 4,
 677,
 4,
 4,
 4,
 4,
 4,
 63,
 4,
 4,
 5,
 4,
 4,
 4,
 1075,
 4,
 335,
 4,
 4,
 4,
 660,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 963,
 4,
 4,
 4,
 4,
 4,
 4,
 808,
 4,
 4,
 4,
 5,
 1778,
 4,
 4,
 4,
 4,
 36,
 4,
 4,
 4,
 294,
 4,
 4,
 4,
 1017,
 4,
 4,
 4,
 968,
 679,
 1769,
 1493,
 855,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 1699,
 4,
 4,
 4,
 4,
 4,
 4,
 47,
 4,
 329,
 4,
 1479,
 2417,
 1

In [61]:
y=le.inverse_transform(pred)

In [69]:
sub.columns

Index(['PRODUCT_ID', 'BROWSE_NODE_ID'], dtype='object')

In [72]:
len(train_df),len(y)

(903024, 110775)

In [74]:
d = {'PRODUCT_ID': test_df.PRODUCT_ID, 'BROWSE_NODE_ID': y}
SUBMISSION = pd.DataFrame(data=d)
SUBMISSION.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,4
1,2,122
2,3,4
3,4,4
4,5,4


In [76]:
SUBMISSION.to_csv('First_Submission.csv',index=False)

sequence_length = train_df.shape[1]
filter_sizes = [3,4]
num_filters = 100
drop = 0.4

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)

merged_tensor = concatenate([maxpool_0, maxpool_1], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((2*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
conc = Dense(40)(dropout)
output = Dense(units=6, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(conc)

# this creates a model that includes
model = Model(inputs, output)

opt = Adam(lr=1e-3)
model.compile(loss='categorical_crossentropy',optimizer=opt)

# Fitting Model to the data
callbacks = [EarlyStopping(monitor='val_loss')]
hist_adam = model.fit(X_train, y_train, batch_size=1000, epochs=20, verbose=2, validation_data=(X_val, y_val),
         callbacks=callbacks)

sequences_test=tokenizer.texts_to_sequences(X_test)
X_test2 = pad_sequences(sequences_test,maxlen=X_train.shape[1])

col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

# Predict on train, val and test datasets
pred_train = model.predict(X_train)
pred_test = model.predict(X_test2)
pred_val = model.predict(X_val)

# Emply array to collect AUC scores
AUC = np.zeros((3,6))
AUC


from sklearn import metrics
for i,x in enumerate(col):
    auc = np.array([metrics.roc_auc_score(y_train[:,i], pred_train[:,i]),
                    metrics.roc_auc_score(y_val[:,i], pred_val[:,i]),
                    metrics.roc_auc_score(y_test[x], pred_test[:,i])])
    print(x,"Train AUC:",auc[0],", Val AUC:",auc[1],", Test AUC:",auc[2])
    AUC[:,i] = auc
    
avg_auc = AUC.mean(axis=1)
print("Average Train AUC:",avg_auc[0],", Average Val AUC:",avg_auc[1],", Average Test AUC:",avg_auc[2])